<a href="https://colab.research.google.com/github/manabil/Tensorflow-Advanced-Techniques-Specialization/blob/main/Custom%20and%20Distributed%20Training%20with%20Tensorflow/Week%203/C2_W3_Lab_2_graphs_for_complex_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autograph: Graphs for complex code

In this ungraded lab, you'll go through some of the scenarios from the lesson `Creating graphs for complex code`.

## Imports

In [1]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


As you saw in the lectures, seemingly simple functions can sometimes be difficult to write in graph mode. Fortunately, Autograph generates this complex graph code for us.

- Here is a function that does some multiplication and additon.

In [2]:
a: tf.Variable = tf.Variable(1.0)
b: tf.Variable = tf.Variable(2.0)

@tf.function
def f(x: float, y: float) -> tf.Tensor:
    a.assign(tf.multiply(y, b))
    b.assign_add(tf.multiply(x, a))
    return tf.add(a, b)

print(f(1.0, 2.0))

print(tf.autograph.to_code(f.python_function))

tf.Tensor(10.0, shape=(), dtype=float32)
def tf__f(x: float, y: float) -> tf.Tensor:
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        ag__.converted_call(ag__.ld(a).assign, (ag__.converted_call(ag__.ld(tf).multiply, (ag__.ld(y), ag__.ld(b)), None, fscope),), None, fscope)
        ag__.converted_call(ag__.ld(b).assign_add, (ag__.converted_call(ag__.ld(tf).multiply, (ag__.ld(x), ag__.ld(a)), None, fscope),), None, fscope)
        try:
            do_return = True
            retval_ = ag__.converted_call(ag__.ld(tf).add, (ag__.ld(a), ag__.ld(b)), None, fscope)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



- Here is a function that checks if the sign of a number is positive or not.

In [3]:
@tf.function
def sign(x: tf.Tensor) -> str:
    if x > 0:
        return 'Positive'
    else:
        return 'Negative or zero'

print(f"Sign = {sign(tf.constant(2))}")
print(f"Sign = {sign(tf.constant(-2))}")

print(tf.autograph.to_code(sign.python_function))

Sign = b'Positive'
Sign = b'Negative or zero'
def tf__sign(x: tf.Tensor) -> str:
    with ag__.FunctionScope('sign', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal retval_, do_return
            (do_return, retval_) = vars_

        def if_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = 'Positive'
            except:
                do_return = False
                raise

        def else_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = 'Negative or zero'
            except:
                do_return = False
                raise
        ag__.if_stmt(ag__.ld(x) 

- Here is another function that includes a while loop.

In [12]:
@tf.function
def f(x: tf.Tensor) -> tf.Tensor:
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

print(tf.autograph.to_code(f.python_function))

def tf__f(x: tf.Tensor) -> tf.Tensor:
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body():
            nonlocal x
            ag__.converted_call(ag__.ld(tf).print, (ag__.ld(x),), None, fscope)
            x = ag__.converted_call(ag__.ld(tf).tanh, (ag__.ld(x),), None, fscope)

        def loop_test():
            return ag__.converted_call(ag__.ld(tf).reduce_sum, (ag__.ld(x),), None, fscope) > 1
        ag__.while_stmt(loop_test, loop_body, get_state, set_state, ('x',), {})
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_

- Here is a function that uses a for loop and an if statement.

In [13]:
@tf.function
def sum_even(items: list) -> int:
    s: int = 0
    for c in items:
        if c % 2 > 0:
            continue
        s += c
    return s

print(tf.autograph.to_code(sum_even.python_function))

def tf__sum_even(items):
    with ag__.FunctionScope('sum_even', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        s = 0

        def get_state_2():
            return (s,)

        def set_state_2(vars_):
            nonlocal s
            (s,) = vars_

        def loop_body(itr):
            nonlocal s
            c = itr
            continue_ = False

            def get_state():
                return (continue_,)

            def set_state(vars_):
                nonlocal continue_
                (continue_,) = vars_

            def if_body():
                nonlocal continue_
                continue_ = True

            def else_body():
                nonlocal continue_
                pass
            ag__.if_stmt(ag__.ld(c) % 2 > 0, if_body, else_body, get_state, set_state, ('continue_',), 1)

            def

## Print statements

Tracing also behaves differently in graph mode. First, here is a function (not decorated with `@tf.function` yet) that prints the value of the input parameter.  `f(2)` is called in a for loop 5 times, and then `f(3)` is called.

In [16]:
def f(x: int) -> None:
    print("Traced with", x)

for i in range(5):
    f(2)

f(3)

Traced with 2
Traced with 2
Traced with 2
Traced with 2
Traced with 2
Traced with 3


If you were to decorate this function with `@tf.function` and run it, notice that the print statement only appears once for `f(2)` even though it is called in a loop.

In [18]:
@tf.function
def f(x: int) -> None:
    print("Traced with", x)

for i in range(5):
    f(2)

f(3)

Traced with 2
Traced with 3


Now compare `print` to `tf.print`.
- `tf.print` is graph aware and will run as expected in loops.

Try running the same code where `tf.print()` is added in addition to the regular `print`.
- Note how `tf.print` behaves compared to `print` in graph mode.

In [19]:
@tf.function
def f(x: int) -> None:
    print("Traced with", x)
    # added tf.print
    tf.print("Executed with", x)

for i in range(5):
    f(2)

f(3)

Traced with 2
Executed with 2
Executed with 2
Executed with 2
Executed with 2
Executed with 2
Traced with 3
Executed with 3


## Avoid defining variables inside the function

This function (not decorated yet) defines a tensor `v` and adds the input `x` to it.  

Here, it runs fine.

In [21]:
def f(x: int) -> tf.Variable:
    v: tf.Variable = tf.Variable(1.0)
    v.assign_add(x)
    return v

print(f(1))

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>


Now if you decorate the function with `@tf.function`.

The cell below will throw an error because `tf.Variable` is defined within the function. The graph mode function should only contain operations.

In [22]:
@tf.function
def f(x: int) -> tf.Variable:
    v: tf.Variable = tf.Variable(1.0)
    v.assign_add(x)
    return v

print(f(1))

ValueError: ignored

To get around the error above, simply move `v = tf.Variable(1.0)` to the top of the cell before the `@tf.function` decorator.

In [23]:
# define the variables outside of the decorated function
v: tf.Variable = tf.Variable(1.0)

@tf.function
def f(x: int) -> tf.Variable:
    return v.assign_add(x)

print(f(5))

tf.Tensor(6.0, shape=(), dtype=float32)
